1. DB연동

In [1]:
from neo4j import GraphDatabase
import requests
from py2neo import Graph
from neo4j import __version__ as neo4j_version

url = "neo4j://172.16.200.28:7687"
user = "neo4j"
password = "testneo4j"

graph = Graph(uri=url, user=user, password=password)

2. 데이터 불러오기

In [2]:
import pandas as pd
object_detail_data = pd.read_json('datasets/data_tools/VG/objects_details.json')
obj_dat = object_detail_data.head()
obj_dat

,synsets,h,object_id,merged_object_ids,names,w,y,x
0,[tree.n.01],557,1058549,[],[trees],799,0,0
1,[sidewalk.n.01],290,1058534,[5046],[sidewalk],722,308,78
2,[building.n.01],538,1058508,[],[building],222,0,1
3,[street.n.01],258,1058539,[3798578],[street],359,283,439
4,[wall.n.01],535,1058543,[],[wall],135,1,0


3. 쿼리 작동 (노드 or 엣지 load)

In [4]:
query = """
    MERGE (o:test2 {{name: "{object_name}", id: "{object_id}"}}) 
    RETURN o.name;
    """

k = 0
for d in obj_dat.values: 
    result = graph.run(query.format(object_name=d[4], object_id=d[2]))
    d = result.data()
    print("row ", k)
    print(d)
    k = k + 1

row  0
[{'o.name': "['trees']"}]
row  1
[{'o.name': "['sidewalk']"}]
row  2
[{'o.name': "['building']"}]
row  3
[{'o.name': "['street']"}]
row  4
[{'o.name': "['wall']"}]


In [ ]:
## 관계 생성
query = """
    MATCH (s:object), (o:object) 
    WHERE s.object_id = {subj_id} and o.object_id = {obj_id}
    CREATE (s)-[r:`{predicate}` {{id: "{rels_id}", synset: "{rels_synset}"}}]->(o) 
    RETURN s.names, o.names, type(r) as relation_type;"""

k = 0
for d in rels_df.values: 
    result = graph.run(query.format(predicate=d[0], rels_id=d[1], rels_synset=d[2], subj_id = d[12], obj_id = d[5]))
    d = result.data()
    print("row ", k)
    print(d)
    k = k + 1

4. 노드 검색하기

In [5]:
q = 'MATCH (n:test2) RETURN n'
results = graph.run(q)
results = list(results)

results

[Record({'n': Node('test2', id='2', name='4')}),
 Record({'n': Node('test2', id='1058549', name="['trees']")}),
 Record({'n': Node('test2', id='1058534', name="['sidewalk']")}),
 Record({'n': Node('test2', id='1058508', name="['building']")}),
 Record({'n': Node('test2', id='1058539', name="['street']")}),
 Record({'n': Node('test2', id='1058543', name="['wall']")})]